In [6]:
import hashlib
import random
from ecdsa import SigningKey, SECP256k1
import datetime
import copy

In [4]:
class User:
    def __init__(self, name, balance):
        self.name = name
        self.generate_keys()
        self.balance = balance
    
    def generate_keys(self):
        self.private_key = SigningKey.generate(curve=SECP256k1)
        self.public_key = self.private_key.get_verifying_key()
        self.public_address = hashlib.sha256(self.public_key.to_string()).hexdigest()[:16]
        return True
    
    def sign_transaction(self, message : bytes) -> bytes:
        return self.private_key.sign(message)

    def __str__(self):
        return f"Name : {self.name}, Public Address : {self.public_address}, Balance : {self.balance}"

In [5]:
class Transaction:
    def __init__(self, from_account, to_account, amount):
        self.from_user = from_account
        self.to_user = to_account
        self.amount = amount
        self.timestamp = datetime.datetime.now()
        self.status = "pending"
        self.trans_hash = self.generate_id()
        self.signature = self.generate_signature()
    
    def generate_id(self):
        hashstr = f"{self.from_user.public_address}{self.to_user.public_address}{self.amount}{self.timestamp}"
        trans_hash = hashlib.sha256(hashstr.encode()).hexdigest()
        return trans_hash
    
    def generate_signature(self):
        message = f"{self.from_user.public_address}{self.to_user.public_address}{self.amount}{self.timestamp}"
        message_bytes = message.encode()
        return self.from_user.sign_transaction(message_bytes)
    
    def complete(self):
        self.status = "complete"
        return True
    
    def fail(self):
        self.status = "failed"
        return False
    
    def verify_signature(self):
        message = f"{self.from_user.public_address}{self.to_user.public_address}{self.amount}{self.timestamp}"
        message_bytes = message.encode()
        try:
            return self.from_user.public_key.verify(self.signature, message_bytes)
        except:
            return False

    def __str__(self):
        return f"Transaction: {self.trans_hash}, From: {self.from_user.public_address}, To: {self.to_user.public_address}, \
            Amount: {self.amount}, Time: {self.timestamp}"

In [ ]:
class Block:
    def __init__(self, transaction_list, prev_hash):
        if len(transaction_list) != 10:
            raise ValueError("Need exactly 10 transactions to create a block.") 
        self.transactions = transaction_list
        self.timestamp = datetime.datetime.now()
        self.nonce = None
        self.hash = None
        self.merkle_root = None
        self.prev_hash = prev_hash
    
    def verify_transactions(self):
        temp_balances = {}
        for tx in self.transactions:
            sender = tx.from_user
            address = sender.public_address
            balance = tx.from_user.balance
            current_balance = temp_balances.get(address, sender.balance)
            if not tx.verify_signature() or tx.amount > current_balance:
                tx.fail()
                return False
            temp_balances[address] = current_balance - tx.amount
        return True
        
    def generate_merkle_root(self):
        tx_hashes = [tx.trans_hash for tx in self.transactions]
        if len(tx_hashes) % 2 == 1:
            tx_hashes.append(tx_hashes[-1])
        while len(tx_hashes) > 1:
            new_layer = []
            for i in range(0, len(tx_hashes), 2):
                combined = (tx_hashes[i] + tx_hashes[i+1]).encode()
                new_hash = hashlib.sha256(combined).hexdigest()
                new_layer.append(new_hash)
            if len(new_layer) % 2 == 1 and len(new_layer) > 1:
                new_layer.append(new_layer[-1])
            tx_hashes = new_layer
        return tx_hashes[0]
    
    def generate_hash(self):
        merkle_root = self.merkle_root
        hashstr = f"{merkle_root}{self.timestamp}{self.prev_hash}{self.nonce}"
        return hashlib.sha256(hashstr.encode()).hexdigest()
    
    def mine(self, difficulty):
        self.merkle_root = self.generate_merkle_root()  # compute once
        if self.verify_transactions():
            prefix = "0" * difficulty
            while True:
                self.nonce = random.getrandbits(32)
                new_hash = self.generate_hash()
                if new_hash.startswith(prefix):
                    self.hash = new_hash
                    for tx in self.transactions:
                        tx.complete()
                    print("Mining done!")
                    return new_hash
        else:
            raise ValueError("Some transactions couldnt be verified")
    
    def verify(self, difficulty):
        recalculated_merkle = self.generate_merkle_root()
        recalculated_hash = hashlib.sha256(
            f"{recalculated_merkle}{self.timestamp}{self.prev_hash}{self.nonce}".encode()
        ).hexdigest()
        prefix = "0" * difficulty
        return recalculated_hash.startswith(prefix) and self.hash == recalculated_hash and self.verify_transactions()
    
    def __str__(self):
        return f"Block hash: {self.hash}, Previous hash: {self.prev_hash}, Block nonce: {self.nonce}\
                Block merkle root: {self.merkle_root}, Block Time stamp: {self.timestamp}"
            

In [40]:
class BlockChain:
    def __init__(self, blocks, transactions):
        self.blocks = blocks
        self.mempool = transactions
        self.ledger = {}
    
    def add_transaction(self, transaction):
        sender = transaction.from_user
        sender_balance = self.ledger.get(sender, 0)
        pending_total = sum(
                tx.amount for tx in self.mempool
                if tx.from_user.public_address == sender
            )
        if sender_balance >= transaction.amount + pending_total:
            if transaction.verify_signature():
                self.mempool.append(transaction)
                return True
            else:
                print("Signature verification failed.")
                return False
        else:
            print("Double spending attempt blocked: insufficient balance in mempool.")
            return False
    
    def add_block(self, block):
        if block.verify(3) and block.prev_hash == self.blocks[-1].hash:
            self.blocks.append(block)
            self.update_ledger(block)
            return True
        return False
    
    def update_ledger(self, block):
        for tx in block.transactions:
            sender = tx.from_user.public_address
            recipient = tx.to_user.public_address
            if sender not in self.ledger:
                self.ledger[sender] = tx.from_user.balance
            if recipient not in self.ledger:
                self.ledger[recipient] = tx.to_user.balance
            if self.ledger[sender] >= tx.amount:
                self.ledger[sender] -= tx.amount
                self.ledger[recipient] += tx.amount
            else:
                print("Ledger inconsistency detected — should not happen!")
                return False
        return True

    def create_block(self):
        if self.blocks:
            hash1 = self.blocks[-1].hash
        else:
            hash1 = None
        if len(self.mempool) >= 10:
            block1 = Block(self.mempool[:10], hash1)
            del self.mempool[:10]
            block1.mine(3)
            self.add_block(block1)
            print(f"Block {block1.hash} created and added to the block chain!")
            return True
        else:
            print(f"Block couldnt be created yet. Need {10 - len(self.mempool)} transactions to create one!")
            return False
     
    def validate(self):
        for i in range(0, len(self.blocks)-1):
            if not(self.blocks[i].timestamp < self.blocks[i+1].timestamp and self.blocks[i].hash == self.blocks[i+1].prev_hash and self.blocks[i].verify(3)):
                print("Block has been tampered!!!")
                return False
        return True
    
    def __str__(self):
        result = "Blocks in the blockchain:\n"
        for i in self.blocks:
            result += f"Block {i.hash}\n"
        result += "Current mempool: \n"
        for j in self.mempool:
            result += f"Transaction {j.trans_hash}, status : {j.status}\n"
        result += "Ledger: \n"
        for user, balance in zip(self.ledger.keys(), self.ledger.values()):
            result += f"User {user}: Balance {balance}\n"
        return result

In [57]:
alice = User("Alice", 100)
bob = User("Bob", 50)
charlie = User("Charlie", 0)

In [58]:
print(bob)

Name : Bob, Public Address : 6c6fd34557f6c5e5, Balance : 50


In [59]:
genesis_block = Block([Transaction(alice, bob, 1) for _ in range(10)], prev_hash="0"*64)
genesis_block.merkle_root = genesis_block.generate_merkle_root()
genesis_block.hash = genesis_block.generate_hash()

In [60]:
blockchain = BlockChain(blocks=[genesis_block], transactions=[])
blockchain.update_ledger(genesis_block)  # initialize ledger

True

In [61]:
blockchain.ledger

{'3e68267f70d704a1': 90, '6c6fd34557f6c5e5': 60}

In [62]:
print(blockchain)

Blocks in the blockchain:
Block 0750e6660ae433dcbd443abfd7a8b7a2107af136b67293c4440bd7b95a02cc26
Current mempool: 
Ledger: 
User 3e68267f70d704a1: Balance 90
User 6c6fd34557f6c5e5: Balance 60



In [63]:
tx1 = Transaction(alice, bob, 20)
blockchain.add_transaction(tx1)

tx2 = Transaction(alice, bob, 0)  # Should fail due to insufficient funds
blockchain.add_transaction(tx2)

tx3 = Transaction(bob, charlie, 30)
blockchain.add_transaction(tx3)

# Add 7 more dummy transactions to reach 10
for _ in range(7):
    tx = Transaction(bob, charlie, 1)
    blockchain.add_transaction(tx)

Double spending attempt blocked: insufficient balance in mempool.
Double spending attempt blocked: insufficient balance in mempool.
Double spending attempt blocked: insufficient balance in mempool.
Double spending attempt blocked: insufficient balance in mempool.
Double spending attempt blocked: insufficient balance in mempool.
Double spending attempt blocked: insufficient balance in mempool.
Double spending attempt blocked: insufficient balance in mempool.
Double spending attempt blocked: insufficient balance in mempool.
Double spending attempt blocked: insufficient balance in mempool.


In [64]:
blockchain.create_block()

Block couldnt be created yet. Need 9 transactions to create one!


False